<a href="https://colab.research.google.com/github/EPRADDH/NLP_Natural_Language_Processing_Methods/blob/main/Copy_of_RNNs__classify__tweets_as_positive_or_negative_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import all libraries
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers.convolutional import Conv1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import pandas as pd
import numpy as np
import spacy
#nlp=spacy.load("en")
nlp = spacy.load('en_core_web_sm',disable=['parser', 'tagger','ner'])

In [ ]:
nlp = spacy.load('en_core_web_sm')

In [ ]:
#load the dataset
train=pd.read_csv("/content/drive/MyDrive/NLP-Natural-Language-Processing-Methods/New folder/training.1600000.processed.noemoticon.csv" , encoding= "latin-1")
Y_train = train[train.columns[0]]
X_train = train[train.columns[5]]

In [ ]:
train.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [ ]:
Y_train.unique()

array([0, 4])

In [ ]:
X_train.head()

0    is upset that he can't update his Facebook by ...
1    @Kenichan I dived many times for the ball. Man...
2      my whole body feels itchy and like its on fire 
3    @nationwideclass no, it's not behaving at all....
4                        @Kwesidei not the whole crew 
Name: @switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D, dtype: object

In [ ]:
# split the data into test and train
from sklearn.model_selection import train_test_split
trainset1x, trainset2x, trainset1y, trainset2y = train_test_split(X_train.values, Y_train.values, test_size=0.02,random_state=42 )
trainset2y=pd.get_dummies(trainset2y)

In [ ]:
trainset2y.head(10)

,0,4
0,0,1
1,1,0
2,1,0
3,1,0
4,0,1
5,0,1
6,0,1
7,1,0
8,0,1
9,1,0


In [ ]:
# function to remove stopwords
def stopwords(sentence):
    
    new=[]
    sentence=nlp(sentence)
    
    for w in sentence:
        if (w.is_stop == False) & (w.pos_ !="PUNCT"):
            new.append(w.string.strip())
        c=" ".join(str(x) for x in new)
    return c

In [ ]:
new=[]
sentence=nlp(sentence)
    
for w in sentence:
    if (w.is_stop == False) & (w.pos_ !="PUNCT"):
        new.append(w.string.strip())
        c=" ".join(str(x) for x in new)
return c

NameError: ignored

In [ ]:
# function to lemmatize the tweets
def lemmatize(sentence):
    sentence=nlp(sentence)
    str=""
    for w in sentence:
        str+=" "+w.lemma_
    return nlp(str)

In [ ]:
#loading the glove model
def loadGloveModel(gloveFile):
    print("Loading Glove Model")
    f = open(gloveFile,'r',encoding='utf-8',errors='ignore')
    model = {}
    for line in f:
        splitLine = line.split()
        word = splitLine[0]
        embedding = [float(val) for val in splitLine[1:]]
        model[word] = embedding
    print ("Done."),len(model),(" words loaded!")
    return model

In [ ]:
model=loadGloveModel('/content/drive/MyDrive/NLP-Natural-Language-Processing-Methods/New folder/glove.twitter.27B.50d.txt')
#gloveFile = 'E:/pre-trained word vectors/glove.twitter.27B.200d.txt'
#use pre-trained word vectors
#https://github.com/3Top/word2vec-api#where-to-get-a-pretrained-models


Loading Glove Model
Done.


In [ ]:
print (len(model))

1193514


In [ ]:
#vectorising the sentences
def sent_vectorizer(sent, model):
    sent_vec = np.zeros(200)
    numw = 0
    for w in sent.split():
        try:
            sent_vec = np.add(sent_vec, model[str(w)])
            numw+=1
        except:
            pass
    return sent_vec


In [ ]:
#obtain a clean vector
cleanvector=[]
for i in range(trainset2x.shape[0]):
    document=trainset2x[i]
    document=document.lower()
    document=lemmatize(document)
    document=str(document)
    cleanvector.append(sent_vectorizer(document,model))


In [ ]:
#Getting the input and output in proper shape
cleanvector=np.array(cleanvector)

cleanvector

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
cleanvector =cleanvector.reshape(len(cleanvector),200,1)

In [ ]:
cleanvector 

array([[[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       ...,

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]]])

In [ ]:
#tokenizing the sequences
tokenizer = Tokenizer(num_words=16000)
tokenizer.fit_on_texts(trainset2x)
sequences = tokenizer.texts_to_sequences(trainset2x)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
data = pad_sequences(sequences, maxlen=15, padding="post")
print(data.shape)

Found 1873 unique tokens.
(400, 15)


In [ ]:
#reshape the data and preparing to train
data=data.reshape(len(cleanvector),15,1)
from sklearn.model_selection import train_test_split
trainx, validx, trainy, validy = train_test_split(data, trainset2y, test_size=0.3,random_state=42 )

In [ ]:
#calculate the number of words
nb_words=len(tokenizer.word_index)+1

In [ ]:
#obtain theembedding matrix
embedding_matrix = np.zeros((nb_words, 50))
for word, i in word_index.items():
    embedding_vector = model.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
print('Null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))

Null word embeddings: 346


In [ ]:
trainy=np.array(trainy)
validy=np.array(validy)

In [ ]:
#building a simple RNN model
def modelbuild():
    model = Sequential()
    model.add(keras.layers.InputLayer(input_shape=(15,1)))
    keras.layers.embeddings.Embedding(nb_words, 15, weights=[embedding_matrix], input_length=15,
    trainable=False)
 
    model.add(keras.layers.recurrent.SimpleRNN(units = 100, activation='relu',
    use_bias=True))
    model.add(keras.layers.Dense(units=1000, input_dim = 2000, activation='sigmoid'))
    model.add(keras.layers.Dense(units=500, input_dim=1000, activation='relu'))
    model.add(keras.layers.Dense(units=2, input_dim=500,activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
#compiling the model
finalmodel = modelbuild()
finalmodel.fit(trainx, trainy, epochs=300, batch_size=64,validation_data=(validx,validy),)

Epoch 1/300
5/5 [==============================] - 0s 48ms/step - loss: 2.2155 - accuracy: 0.5357 - val_loss: 1.5015 - val_accuracy: 0.4917
Epoch 2/300
5/5 [==============================] - 0s 12ms/step - loss: 1.2767 - accuracy: 0.4643 - val_loss: 0.8950 - val_accuracy: 0.5000
Epoch 3/300
5/5 [==============================] - 0s 11ms/step - loss: 0.8700 - accuracy: 0.5357 - val_loss: 0.7420 - val_accuracy: 0.4917
Epoch 4/300
5/5 [==============================] - 0s 12ms/step - loss: 0.6620 - accuracy: 0.5821 - val_loss: 0.7745 - val_accuracy: 0.5167
Epoch 5/300
5/5 [==============================] - 0s 12ms/step - loss: 0.6792 - accuracy: 0.5500 - val_loss: 0.7306 - val_accuracy: 0.5083
Epoch 6/300
5/5 [==============================] - 0s 13ms/step - loss: 0.6563 - accuracy: 0.5786 - val_loss: 0.7323 - val_accuracy: 0.4833
Epoch 7/300
5/5 [==============================] - 0s 13ms/step - loss: 0.6154 - accuracy: 0.6964 - val_loss: 0.7442 - val_accuracy: 0.5500
Epoch 8/300
5/5 [===

In [ ]:
glove = pd.read_csv('/content/drive/MyDrive/NLP-Natural-Language-Processing-Methods/New folder/glove.twitter.27B.50d.txt')

ParserError: ignored